In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# # Install nessecary libraries
!pip install transformers>=4.48.0
!pip install speechbrain


## Dataset preparation

In [ ]:
import os

# Convert .m4a audio to wav
def save_wav(path, f_path):
  os.makedirs(f_path, exist_ok=True)

  for i, m4a_file in enumerate(sorted(os.listdir(path))):
      if m4a_file.endswith(".m4a"):
          input_path = os.path.join(path, m4a_file)
          out = ['1']
          for j,c in enumerate(m4a_file):
            # Get the number between paranthleses for renaming the wav to just number
            if c=="(":
              out = m4a_file[j+1]
              for temp in m4a_file[j+2:]:
                if temp==')':
                  out = [out]
                  break
                out = out+temp

              if not out:
                out = [m4a_file[j+1]]

          output_path = os.path.join(f_path, f"{'000000'+out[0]}.wav")


          command = f'ffmpeg -i "{input_path}" -ac 1 -ar 16000 "{output_path}"'
          os.system(command)

          print(f"Transformed: {m4a_file} -> {os.path.basename(output_path)}")

f_path = "drive/My Drive/audiobooks/wav/"
path = "drive/My Drive/audiobooks/one_sentence/"
save_wav(path, f_path)
f_path = "drive/My Drive/audiobooks/wav/"
path = "drive/My Drive/audiobooks/two_sentence/"
save_wav(path, f_path)


In [3]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import csv

# Convert data to dataframe - text (transcript), audio
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def split_sentences(text, language='czech'):
    return sent_tokenize(text, language=language)

def group_sentences(sentences, group_size):
    return [' '.join(sentences[i:i+group_size]) for i in range(0, len(sentences), group_size)]

def save_metadata(segments, output_path='drive/My Drive/audiobooks/metadata.csv',len_of_the_first_segments=0):
    with open(output_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile, delimiter='|')
        for idx, segment in enumerate(segments):
            wav_filename = f"{f'000000{idx+1+len_of_the_first_segments}'}.wav" # I filtered out 40 between 133 and 172.wav
            writer.writerow([wav_filename, segment])
    print(f"metadata.csv uloženo s {len(segments)} záznamy.")
def concatenate_non_uppercase(lst):
    if not lst:
        return []

    result = [lst[0]]
    # concatenate to previous string if not uppercase the next letter
    for s in lst[1:]:
        if s and not s[0].isupper():
            result[-1] += ' ' + s
        else:
            result.append(s)
    return result

txt_path = "drive/My Drive/audiobooks/one_sentence/transcript_part_1.txt"
group_size = 1         # one or mor sentences
text = load_text(txt_path)

# Replace some symbols
text = text.replace("\n", " ")
text = text.replace('“','')
text = text.replace('„','')
text = text.replace('...','')

print(text)
sentences = split_sentences(text)
sentences_copy=concatenate_non_uppercase(sentences)
segments = group_sentences(sentences_copy, group_size)
save_metadata(segments,output_path='drive/My Drive/audiobooks/metadata_one.csv')
print(len(segments))
len_of_the_first_segments = len(segments)
segments

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


A vaše jméno?  Nejmenuji se; jsem nikdo. Jsem hůř než nikdo, jsem  krysař.  Muž, který takto mluvil, stál se vzpřímenou hlavou před vraty  domu, v nichž se do soumraku bělala ženská postava. Hleděl  na ni svýma temnýma, pátravýma očima. Byl vysoký a štíhlý,  štíhlejší ještě ve svém přiléhavém sametovém kabátci  a v úzkých nohavicích. Jeho ruce byly drobné a jemné jako  ruce paní. Neměl při sobě ani zbraně ani hole, ač se zdálo, že  přichází z daleka, po cestách, jež vždy jisty nebyly. Svíral zato  něco dlouhého a ozdobného, co budilo zvědavost ženy, s kterou  hovořil. Byla to píšťala cizokrajné práce, jaké dosud neviděla.  Krysař, zasmála se žena ve dveřích. Přicházíte včas  do Hammeln. Není tu krysaře; zato krys je tu mnoho.  Vysvětlete mi, krysaři, odkud se berou krysy? Nebývalo jich  dříve, jak nám říkají. Je pravda, končila s úsměvem, staří lidé  míní, že svět je stále horší.  Krysař pokrčil rameny.  Odkud jsou, nevím. Jsou však v každém z vašich domů.  Hlodají bez ustání; hlodají 

['A vaše jméno?',
 'Nejmenuji se; jsem nikdo.',
 'Jsem hůř než nikdo, jsem  krysař.',
 'Muž, který takto mluvil, stál se vzpřímenou hlavou před vraty  domu, v nichž se do soumraku bělala ženská postava.',
 'Hleděl  na ni svýma temnýma, pátravýma očima.',
 'Byl vysoký a štíhlý,  štíhlejší ještě ve svém přiléhavém sametovém kabátci  a v úzkých nohavicích.',
 'Jeho ruce byly drobné a jemné jako  ruce paní.',
 'Neměl při sobě ani zbraně ani hole, ač se zdálo, že  přichází z daleka, po cestách, jež vždy jisty nebyly.',
 'Svíral zato  něco dlouhého a ozdobného, co budilo zvědavost ženy, s kterou  hovořil.',
 'Byla to píšťala cizokrajné práce, jaké dosud neviděla.',
 'Krysař, zasmála se žena ve dveřích.',
 'Přicházíte včas  do Hammeln.',
 'Není tu krysaře; zato krys je tu mnoho.',
 'Vysvětlete mi, krysaři, odkud se berou krysy?',
 'Nebývalo jich  dříve, jak nám říkají.',
 'Je pravda, končila s úsměvem, staří lidé  míní, že svět je stále horší.',
 'Krysař pokrčil rameny.',
 'Odkud jsou, nevím.

In [4]:
txt_path = "drive/My Drive/audiobooks/two_sentence/transcript_2_vety.txt"
group_size = 2         # one or mor sentences
text = load_text(txt_path)

# Replace some symbols
text = text.replace("\n", " ")
text = text.replace('“','')
text = text.replace('„','')
text = text.replace('...','')

print(text)
sentences = split_sentences(text)
sentences_copy=concatenate_non_uppercase(sentences)
segments = group_sentences(sentences_copy, group_size)
save_metadata(segments,output_path='drive/My Drive/audiobooks/metadata_two.csv',len_of_the_first_segments=len_of_the_first_segments)
segments

Byl to kout v dobrém hansovním městě Hammeln, tichý a opuštěný kout, skoro,  jako by tu nebylo město. Těžké vozy se zbožím tudy nehrčely; nehlučely tu trhy a  nejezdili jezdci v bohaté zbroji. Ani procesí tudy nechodila. Pouze hlas zvonu sem  zněl z chrámu svaté Trojice v Hammeln; hlas vážný a zádumčivý. Ale ty hlasy,  které jsme tak často slyšeli, přestáváme vnímat. Aspoň Agnes, otevřela-li své  okénko, slyšela spíše jiné hlasy.  Pod jejími okny byla zahrada v květu, kde zpívali ptáci, vše bzučelo životem a  úsměvy na její pozdrav. V zahradě bylo tolik vůní, které opojovaly. A Agnes se  zdálo, že je květem mezi květy a vůní mezi vůněmi. Byl tu také dům s prejzovou střechou, starý a skrytý v zeleni kaštanů. Ale tento  dům uměl náhle zjasnět a otevřít se slunci.  Matka krásné Agnes byla žena znavená spíše smutnými příběhy, nežli věkem.  Na tyto příběhy vzpomínala s hrůzou a steskem, zimomřivě se chvěla a ploužila se  domem jako stín. Slunce a světlo ji lekalo; prchala před ním jako nočn

['Byl to kout v dobrém hansovním městě Hammeln, tichý a opuštěný kout, skoro,  jako by tu nebylo město. Těžké vozy se zbožím tudy nehrčely; nehlučely tu trhy a  nejezdili jezdci v bohaté zbroji.',
 'Ani procesí tudy nechodila. Pouze hlas zvonu sem  zněl z chrámu svaté Trojice v Hammeln; hlas vážný a zádumčivý.',
 'Ale ty hlasy,  které jsme tak často slyšeli, přestáváme vnímat. Aspoň Agnes, otevřela-li své  okénko, slyšela spíše jiné hlasy.',
 'Pod jejími okny byla zahrada v květu, kde zpívali ptáci, vše bzučelo životem a  úsměvy na její pozdrav. V zahradě bylo tolik vůní, které opojovaly.',
 'A Agnes se  zdálo, že je květem mezi květy a vůní mezi vůněmi. Byl tu také dům s prejzovou střechou, starý a skrytý v zeleni kaštanů.',
 'Ale tento  dům uměl náhle zjasnět a otevřít se slunci. Matka krásné Agnes byla žena znavená spíše smutnými příběhy, nežli věkem.',
 'Na tyto příběhy vzpomínala s hrůzou a steskem, zimomřivě se chvěla a ploužila se  domem jako stín. Slunce a světlo ji lekalo; prc

In [ ]:
from transformers import SpeechT5Processor
# Load Processor (tokenizor) from SpeechT5
checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

# Print vocab of pretrained model
tokenizer = processor.tokenizer
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}
tokenizer_vocab

In [8]:
import pandas as pd
metadata_path = "drive/My Drive/audiobooks/metadata_one.csv"
df1 = pd.read_csv(metadata_path, delimiter='|',header=None)
df1.columns = ["filename", "text"]

metadata_path = "drive/My Drive/audiobooks/metadata_two.csv"
df2 = pd.read_csv(metadata_path, delimiter='|',header=None)
df2.columns = ["filename", "text"]
df = pd.concat([df1, df2], ignore_index=True)
df


,filename,text
0,0000001.wav,A vaše jméno?
1,0000002.wav,Nejmenuji se; jsem nikdo.
2,0000003.wav,"Jsem hůř než nikdo, jsem krysař."
3,0000004.wav,"Muž, který takto mluvil, stál se vzpřímenou hl..."
4,0000005.wav,"Hleděl na ni svýma temnýma, pátravýma očima."
...,...,...
167,000000168.wav,A tak Kristián kolísal mezi úzkostí a nadějí. ...
168,000000169.wav,Vypravoval jí o svých starostech a bezesných n...
169,000000170.wav,"Její starostí bylo, aby Kristián nevědělo krys..."
170,000000171.wav,Agnes naslouchala a souhlasila; tak si alespoň...


In [9]:
# Print characters that are in text but not in vocab of tokenizer
l = []
for char in df["text"]:
  for c in char:
     l.append(c)
print("All chars: ",set(l))
print("Not in vocab: ",set(l)-tokenizer_vocab)

All chars:  {'č', 'y', 'e', 'x', '.', ',', 'š', 'ů', 'c', 'ž', 'ť', 'd', 'Z', '»', 'M', 'ö', 'r', 'N', 'ú', 'm', 'C', 'f', '–', 'p', 'h', '?', 'D', 'V', 'Č', 'ň', 'A', 't', 'L', 'l', 'H', 'n', ' ', 'a', 'P', 'ř', 'í', 'é', 'J', 'ě', 'R', 'j', 'i', 'v', 'T', 'U', 's', 'Ř', 'G', 'ý', 'b', 'S', '!', 'u', 'z', 'K', 'O', 'Ž', ':', 'o', ';', 'g', 'á', '«', 'B', 'k', '-'}
Not in vocab:  {'č', 'š', 'ů', '»', 'ž', 'ť', 'ö', 'ú', '–', 'Č', 'ň', ' ', 'ř', 'í', 'ě', 'Ř', 'ý', 'Ž', '«', 'á'}


In [10]:
#Map czech letters to english transcription

map_czech_letters = {
    'í': 'ee', # see
    'ř': 'rz', # nothing similar in english
    'Ř': 'RZ', # nothing similar in english
    'ý': 'yy', #
    'ě': 'ye', # yes
    'č': 'cz', # better would be 'ch' but 'ch' is letter in czech
    'Č': 'CZ', # better would be 'ch' but 'ch' is letter in czech
    'ů': 'uu',
    'ú': 'uu',
    'š': 'sh', # shoe
    'Š':'Sh',
    'ď': 'du', # Duke
    'ť': 'tu', # tune
    'ň': 'ny', # canyon
    'ž': 'zh', # zs would be also possible
    'Ž': 'ZH', #
    'á': 'aa',
    'ö': 'o', # I read ö as o in audio (There was name Jörgen)
    '«':'',
    '–':'-',
    '»':'',
}
def replace_czech_letters(text, mapping):
    return ''.join(mapping.get(c, c) for c in text)
df["text_phonetic"] = df["text"].apply(lambda x: replace_czech_letters(x, map_czech_letters))
df

,filename,text,text_phonetic
0,0000001.wav,A vaše jméno?,A vashe jméno?
1,0000002.wav,Nejmenuji se; jsem nikdo.,Nejmenuji se; jsem nikdo.
2,0000003.wav,"Jsem hůř než nikdo, jsem krysař.","Jsem huurz nezh nikdo, jsem krysarz."
3,0000004.wav,"Muž, který takto mluvil, stál se vzpřímenou hl...","Muzh, kteryy takto mluvil, staal se vzprzeemen..."
4,0000005.wav,"Hleděl na ni svýma temnýma, pátravýma očima.","Hledyel na ni svyyma temnyyma, paatravyyma oc..."
...,...,...,...
167,000000168.wav,A tak Kristián kolísal mezi úzkostí a nadějí. ...,A tak Kristiaan koleesal mezi uuzkostee a nady...
168,000000169.wav,Vypravoval jí o svých starostech a bezesných n...,Vypravoval jee o svyych starostech a bezesnyyc...
169,000000170.wav,"Její starostí bylo, aby Kristián nevědělo krys...","Jejee starostee bylo, aby Kristiaan nevyedyelo..."
170,000000171.wav,Agnes naslouchala a souhlasila; tak si alespoň...,Agnes naslouchala a souhlasila; tak si alespon...


In [11]:
# Print again if all characters are in vocab
l = []
for char in df["text_phonetic"]:
  for c in char:
     l.append(c)
print(set(l))
print(set(l)-tokenizer_vocab)

{'y', 'e', '.', ',', 'c', 'd', 'Z', 'M', 'r', 'N', 'f', 'm', 'C', 'p', 'h', '?', 'D', 'V', 'A', 't', 'L', 'l', 'H', 'n', ' ', 'a', 'P', 'x', 'é', 'J', 'R', 'j', 'i', 'v', 'T', 'U', 's', 'G', 'b', 'S', '!', 'u', 'z', 'K', 'O', ':', 'o', ';', 'g', 'B', 'k', '-'}
{' '}


In [ ]:
import numpy as np

# Create audio in dataset of hugging face style so we can convert it
audio_dir = "drive/My Drive/audiobooks/wav"
audio_list = []
transcriptions = []

for _, row in df.iterrows():
    filename = row["filename"]
    text = row["text_phonetic"]
    filename = filename.replace(".wav", "")
    audio_path = os.path.join(audio_dir, f"{filename}.wav")

    sampling_rate, array = wavfile.read(audio_path)
    array = array.astype(np.float16)

    audio_list.append({
        "path": audio_path,
        "array": array,
        "sampling_rate": sampling_rate
    })
    transcriptions.append(text)

dataset_dict = {
    "audio": audio_list,
    "transcription": transcriptions
}

dataset_dict

In [ ]:
import speechbrain
import torch
import torchaudio
from speechbrain.pretrained import EncoderClassifier
# Other pretrained model needed for creating embedding for my voice
model_name = "speechbrain/spkrec-xvect-voxceleb"


device = "cuda" if torch.cuda.is_available() else "cpu"

classifier = EncoderClassifier.from_hparams(
    source=model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", model_name)
)
wav_dir = "drive/My Drive/audiobooks/wav"      # např. "data/wavs"
embeddings = torch.full((1,512), torch.nan)
# Generate embedding - just one is enough for one voice
for i,fname in enumerate(os.listdir(wav_dir)):
    if fname.endswith(".wav"):
        path = os.path.join(wav_dir, fname)

        # Načti zvuk
        signal, fs = torchaudio.load(path)
        if fs != 16000:
            signal = torchaudio.functional.resample(signal, fs, 16000)

        # Přesuň na zařízení
        signal = signal.to(device)

        # Vytvoř embedding
        with torch.no_grad():
            embedding = classifier.encode_batch(signal)
            embedding = torch.nn.functional.normalize(embedding, dim=2)
            embedding = embedding.squeeze().cpu()
            embedding = embedding.to(torch.float16)
            embeddings[i] = embedding
            break
embeddings = embeddings.repeat(len(df),1)
print(embeddings.shape)
dataset_dict["embeddings"] = embeddings


In [14]:
from datasets import Dataset
# Create hugging face dataset

dataset_dict_copy={}
dataset_dict_copy = {
    "input_ids": dataset_dict["transcription"],  # list of lists
    "labels":  dataset_dict["audio"], # Extract just the audio arrays
    "speaker_embeddings": dataset_dict["embeddings"]
}
dataset = Dataset.from_dict(dataset_dict_copy)

In [ ]:
# Preprocess (tokenized, convert to spectrogram) dataset
def prepare_dataset(example):
    audio = example["labels"]
    speaker_emb = example["speaker_embeddings"]

    example = processor(
        text=example["input_ids"],
        audio_target=audio["array"],
        sampling_rate=16000,
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = torch.Tensor(example["labels"][0])
    example["speaker_embeddings"] = speaker_emb
    return example
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

In [ ]:
# Filter sequences with more than 200 tokens (model not ok with that)
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

In [17]:
# Create test (=validation for this project) set
dataset = dataset.train_test_split(test_size=0.1)

## Training

In [18]:
from dataclasses import dataclass

# Batch creator
@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(self, features):
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        del batch["decoder_attention_mask"]

        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch
data_collator = TTSDataCollatorWithPadding(processor)

In [ ]:
# Finally load pretrained model which we are going to fine tune
from transformers import SpeechT5ForTextToSpeech
from functools import partial


model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

# disable cache during training
model.config.use_cache = False
model.generate = partial(model.generate, use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Set some parameters of the trainer

trainer = Seq2SeqTrainer(
    args=Seq2SeqTrainingArguments(output_dir="drive/My Drive/audiobooks",

    per_device_train_batch_size=4,
    learning_rate=1e-4,
    max_steps=2100,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=4,
    save_steps=2100,
    eval_steps=100,
    logging_steps=25,
    load_best_model_at_end=True,
    greater_is_better=False,
    report_to=[],
    remove_unused_columns=False,
    label_names=["labels"],),
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

In [45]:
# Finally train (fine tune)
trainer.train()

Step,Training Loss,Validation Loss
100,0.359200,0.425249
200,0.358000,0.437893
300,0.354600,0.444431
400,0.370900,0.425634
500,0.342700,0.439347
600,0.352100,0.436837
700,0.351100,0.423308
800,0.349500,0.446530
900,0.332900,0.425235
1000,0.345600,0.425453


TrainOutput(global_step=2100, training_loss=0.34127604166666664, metrics={'train_runtime': 718.7586, 'train_samples_per_second': 11.687, 'train_steps_per_second': 2.922, 'total_flos': 799201207763640.0, 'train_loss': 0.34127604166666664, 'epoch': 55.26315789473684})

In [ ]:
# Test model on some sentences if there are similar to my voice
"
final_text = "Chvílemi zněla píšťala tak, jako by krysař přece ještě někoho volal."
final_text = replace_czech_letters(final_text, map_czech_letters)
inputs = processor(text=final_text, return_tensors="pt")

In [55]:
from transformers import SpeechT5HifiGan # Needed for transfering spectrogram to audio

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

model = SpeechT5ForTextToSpeech.from_pretrained( # Load pretrained model
    "drive/My Drive/audiobooks/checkpoint-2000"
)

example = dataset["test"]
speaker_embeddings = torch.tensor(example["speaker_embeddings"])[:1]
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder) # Generate my voice speech


## Test: AI generated voice x My reference voice

In [56]:
from IPython.display import Audio

# My voice created by neural network
Audio(speech.numpy(), rate=16000)

In [6]:
from IPython.display import Audio
from scipy.io import wavfile

rate, data = wavfile.read("drive/My Drive/audiobooks/label.wav")
# Reference audio
Audio(data, rate=rate)
